# Sensitivity Analysis

## Purpose

This notebook teaches you how to perform sensitivity analysis for process optimization. You'll learn global methods (Sobol, Morris, FAST), local methods, Design of Experiments (DoE), and uncertainty quantification with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Perform global sensitivity analysis (Sobol indices, Morris screening, FAST)
- ✅ Use local sensitivity methods (derivatives, gradients)
- ✅ Design experiments for sensitivity analysis
- ✅ Quantify uncertainty in model outputs
- ✅ Rank parameter importance and visualize sensitivity results

## Estimated Duration

60-90 minutes

---

## Overview

Sensitivity analysis identifies which input parameters most influence model outputs. The AM-QADF framework provides comprehensive sensitivity analysis capabilities:

- 🌐 **Global Methods**: Sobol indices, Morris screening, FAST, RBD, Delta, PAWN, DGSM
- 📍 **Local Methods**: Derivatives, gradients, perturbation analysis
- 🧪 **Design of Experiments**: Factorial, LHS, Random, Grid designs
- 📊 **Uncertainty Quantification**: Monte Carlo, Bayesian, Taylor expansion

Use the interactive widgets below to perform sensitivity analysis - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import sensitivity analysis classes
SENSITIVITY_AVAILABLE = False
try:
    from am_qadf.analytics.sensitivity_analysis.client import SensitivityAnalysisClient
    SENSITIVITY_AVAILABLE = True
    print("✅ Sensitivity analysis classes available")
except ImportError as e:
    print(f"⚠️ Sensitivity analysis classes not available: {e} - using demo mode")

# MongoDB connection setup
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None
stl_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    from am_qadf.query import STLModelClient
    
    # Initialize MongoDB connection
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    if mongo_client.is_connected():
        voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
        stl_client = STLModelClient(mongo_client=mongo_client)
        INFRASTRUCTURE_AVAILABLE = True
        print(f"✅ Connected to MongoDB: {config.database}")
    else:
        print("⚠️ MongoDB connection failed")
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


✅ Sensitivity analysis classes available


Failed to connect to MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 696012ab8b1670a5cc2c2b5b, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


⚠️ MongoDB not available: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 696012ab8b1670a5cc2c2b5b, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]> - using demo mode
✅ Setup complete!


## Interactive Sensitivity Analysis Interface

Use the widgets below to perform sensitivity analysis. Select method, configure parameters, and visualize sensitivity indices interactively!


In [2]:
# Create Interactive Sensitivity Analysis Interface

# Global state
sensitivity_results = {}
variable_configs = {}
current_model_id = None
current_grid_id = None
loaded_grid_data = None
signal_arrays = {}

# ============================================
# Helper Functions for Demo Data
# ============================================

def demo_model_function(params):
    """Demo model function for sensitivity analysis."""
    # Simple quadratic model with interactions
    x1 = params.get('laser_power', 200)
    x2 = params.get('scan_speed', 1.0)
    x3 = params.get('layer_thickness', 0.05)
    
    # Normalize inputs
    x1_norm = (x1 - 150) / 150  # 150-300 range
    x2_norm = (x2 - 0.5) / 1.5  # 0.5-2.0 range
    x3_norm = (x3 - 0.02) / 0.08  # 0.02-0.1 range
    
    # Model output (quality score)
    output = (
        0.5 * x1_norm +
        0.3 * x2_norm +
        0.2 * x3_norm +
        0.1 * x1_norm * x2_norm +
        0.05 * x2_norm * x3_norm +
        np.random.normal(0, 0.01)  # Small noise
    )
    
    return max(0, min(1, output))  # Clamp to [0, 1]

# ============================================
# Top Panel: Data Source and Grid Selection
# ============================================

# Data source mode
data_source_label = widgets.HTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('Sample Data', 'sample')],
    value='mongodb',
    description='Source:',
    style={'description_width': 'initial'}
)

# Model selection (for MongoDB)
model_label = widgets.HTML("<b>Model:</b>")
model_options = [("━━━ Select Model ━━━", None)]
if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Grid type filter
grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type_filter = Dropdown(
    options=[
        ('All Grids', 'all'),
        ('Fused', 'fused'),
        ('Corrected', 'corrected'),
        ('Processed', 'processed'),
        ('Signal-Mapped', 'signal_mapped'),
        ('Raw', 'raw')
    ],
    value='fused',  # Default to fused grids
    description='Type:',
    style={'description_width': 'initial'}
)

# Grid selection (for MongoDB)
grid_label = widgets.HTML("<b>Grid:</b>")
grid_dropdown = Dropdown(
    options=[("━━━ Select Grid ━━━", None)],
    value=None,
    description='Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='500px')
)

load_grid_button = Button(
    description='Load Grid',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px')
)

# Sensitivity method
sensitivity_method = Dropdown(
    options=[
        ('Sobol', 'sobol'),
        ('Morris', 'morris'),
        ('FAST', 'fast'),
        ('Local', 'local'),
        ('DoE', 'doe'),
        ('Uncertainty Quantification', 'uq')
    ],
    value='sobol',
    description='Method:',
    style={'description_width': 'initial'}
)

# Output model selector (for sensitivity analysis)
output_model_label = widgets.HTML("<b>Output:</b>")
output_model_selector = Dropdown(
    options=[('Quality Score', 'quality'), ('Signal Value', 'signal'), ('Custom', 'custom')],
    value='quality',
    description='Output:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Analysis',
    button_style='success',
    icon='play',
    layout=Layout(width='160px')
)

compare_button = Button(
    description='Compare Methods',
    button_style='',
    icon='copy',
    layout=Layout(width='160px')
)

top_panel = VBox([
    HBox([data_source_label, data_source_mode, sensitivity_method, output_model_label, output_model_selector]),
    HBox([model_label, model_dropdown, grid_type_label, grid_type_filter]),
    HBox([grid_label, grid_dropdown, load_grid_button]),
    HBox([execute_button, compare_button])
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Sensitivity Configuration
# ============================================

# Problem Definition Section
problem_label = widgets.HTML("<b>Problem Definition:</b>")
n_variables = IntSlider(value=3, min=1, max=10, step=1, description='Variables:', style={'description_width': 'initial'})

# Variable configuration (simplified - using fixed variables for demo)
variable_names = ['laser_power', 'scan_speed', 'layer_thickness']
variable_bounds = {
    'laser_power': (150, 300),
    'scan_speed': (0.5, 2.0),
    'layer_thickness': (0.02, 0.1)
}

problem_section = VBox([
    problem_label,
    n_variables,
    widgets.HTML("<p><b>Variables:</b><br>• laser_power: 150-300 W<br>• scan_speed: 0.5-2.0 mm/s<br>• layer_thickness: 0.02-0.1 mm</p>")
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Sobol Analysis Section
sobol_label = widgets.HTML("<b>Sobol Analysis:</b>")
sobol_sample_size = IntSlider(value=1000, min=100, max=10000, step=100, description='Sample Size:', style={'description_width': 'initial'})
sobol_second_order = Checkbox(value=False, description='Compute Second Order', style={'description_width': 'initial'})
sobol_confidence = FloatSlider(value=0.95, min=0.90, max=0.99, step=0.01, description='Confidence:', style={'description_width': 'initial'})

sobol_section = VBox([
    sobol_label,
    sobol_sample_size,
    sobol_second_order,
    sobol_confidence
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Morris Analysis Section
morris_label = widgets.HTML("<b>Morris Analysis:</b>")
morris_sample_size = IntSlider(value=100, min=10, max=1000, step=10, description='Sample Size:', style={'description_width': 'initial'})
morris_levels = IntSlider(value=10, min=4, max=20, step=1, description='Levels:', style={'description_width': 'initial'})
morris_grid_jump = IntSlider(value=2, min=1, max=10, step=1, description='Grid Jump:', style={'description_width': 'initial'})

morris_section = VBox([
    morris_label,
    morris_sample_size,
    morris_levels,
    morris_grid_jump
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Local Analysis Section
local_label = widgets.HTML("<b>Local Analysis:</b>")
local_perturbation = FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='Perturbation:', style={'description_width': 'initial'})
local_method = RadioButtons(
    options=[('Finite Difference', 'finite_diff'), ('Automatic Differentiation', 'auto_diff')],
    value='finite_diff',
    description='Method:',
    style={'description_width': 'initial'}
)

local_section = VBox([
    local_label,
    local_perturbation,
    local_method
], layout=Layout(padding='5px', border='1px solid #ddd'))

# DoE Section
doe_label = widgets.HTML("<b>Design of Experiments:</b>")
doe_design_type = Dropdown(
    options=[('Factorial', 'factorial'), ('LHS', 'lhs'), ('Random', 'random'), ('Grid', 'grid')],
    value='lhs',
    description='Design Type:',
    style={'description_width': 'initial'}
)
doe_sample_size = IntSlider(value=100, min=10, max=1000, step=10, description='Sample Size:', style={'description_width': 'initial'})
doe_levels = IntSlider(value=3, min=2, max=10, step=1, description='Levels:', style={'description_width': 'initial'})

doe_section = VBox([
    doe_label,
    doe_design_type,
    doe_sample_size,
    doe_levels
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Uncertainty Quantification Section
uq_label = widgets.HTML("<b>Uncertainty Quantification:</b>")
uq_method = RadioButtons(
    options=[('Monte Carlo', 'monte_carlo'), ('Bayesian', 'bayesian'), ('Taylor', 'taylor')],
    value='monte_carlo',
    description='Method:',
    style={'description_width': 'initial'}
)
uq_sample_size = IntSlider(value=1000, min=100, max=10000, step=100, description='Sample Size:', style={'description_width': 'initial'})
uq_distribution = Dropdown(
    options=[('Uniform', 'uniform'), ('Normal', 'normal'), ('Custom', 'custom')],
    value='uniform',
    description='Distribution:',
    style={'description_width': 'initial'}
)

uq_section = VBox([
    uq_label,
    uq_method,
    uq_sample_size,
    uq_distribution
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Show/hide sections based on method
def update_method_sections(change):
    """Show/hide method sections based on selected method."""
    method = change['new']
    sobol_section.layout.display = 'none'
    morris_section.layout.display = 'none'
    local_section.layout.display = 'none'
    doe_section.layout.display = 'none'
    uq_section.layout.display = 'none'
    
    if method == 'sobol':
        sobol_section.layout.display = 'flex'
    elif method == 'morris':
        morris_section.layout.display = 'flex'
    elif method == 'local':
        local_section.layout.display = 'flex'
    elif method == 'doe':
        doe_section.layout.display = 'flex'
    elif method == 'uq':
        uq_section.layout.display = 'flex'

sensitivity_method.observe(update_method_sections, names='value')
update_method_sections({'new': sensitivity_method.value})

# ============================================
# Helper Functions for MongoDB
# ============================================

def update_grid_dropdown(change=None):
    """Update grid dropdown when model or grid type changes."""
    global current_model_id
    
    model_id = model_dropdown.value
    grid_type = grid_type_filter.value
    
    if not model_id:
        grid_dropdown.options = [("━━━ Select Grid ━━━", None)]
        return
    
    current_model_id = model_id
    
    if not voxel_storage:
        grid_dropdown.options = [("━━━ MongoDB not available ━━━", None)]
        return
    
    try:
        # Get all grids for this model
        grids = voxel_storage.list_grids(model_id=model_id, limit=100)
        
        grid_options = [("━━━ Select Grid ━━━", None)]
        for grid in grids:
            metadata = grid.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Determine grid type
            is_fused = config_meta.get('fusion_applied', False)
            is_corrected = config_meta.get('correction_applied', False)
            is_processed = config_meta.get('processing_applied', False)
            has_signals = len(grid.get('available_signals', [])) > 0
            
            grid_type_match = False
            if grid_type == 'all':
                grid_type_match = True
            elif grid_type == 'fused' and is_fused:
                grid_type_match = True
            elif grid_type == 'corrected' and is_corrected:
                grid_type_match = True
            elif grid_type == 'processed' and is_processed:
                grid_type_match = True
            elif grid_type == 'signal_mapped' and has_signals and not is_corrected and not is_processed and not is_fused:
                grid_type_match = True
            elif grid_type == 'raw' and not has_signals:
                grid_type_match = True
            
            if grid_type_match:
                grid_id = grid.get('grid_id', str(grid.get('_id', '')))
                grid_name = grid.get('grid_name', 'Unknown')
                n_signals = len(grid.get('available_signals', []))
                
                # Build status label
                status_parts = []
                if is_fused:
                    status_parts.append('fused')
                if is_corrected:
                    status_parts.append('corrected')
                if is_processed:
                    status_parts.append('processed')
                if has_signals and not status_parts:
                    status_parts.append('mapped')
                if not status_parts:
                    status_parts.append('raw')
                
                status_str = ', '.join(status_parts)
                label = f"{grid_name} ({n_signals} signal(s), {status_str}) ({grid_id[:8]}...)"
                grid_options.append((label, grid_id))
        
        if len(grid_options) == 1:
            grid_options.append(("No grids found matching filter", None))
        
        grid_dropdown.options = grid_options
    except Exception as e:
        grid_dropdown.options = [("━━━ Error loading grids ━━━", None)]
        print(f"⚠️ Error loading grids: {e}")

def load_grid_from_mongodb(button):
    """Load selected grid from MongoDB."""
    global current_model_id, current_grid_id, loaded_grid_data, signal_arrays
    
    if not voxel_storage or not grid_dropdown.value:
        status_display.value = "<b>Status:</b> <span style='color: red;'>⚠️ Please select a grid to load</span>"
        return
    
    grid_id = grid_dropdown.value
    current_grid_id = grid_id
    
    status_display.value = "<b>Status:</b> Loading grid from MongoDB..."
    progress_bar.value = 0
    
    try:
        # Load grid from MongoDB
        grid_data = voxel_storage.load_voxel_grid(grid_id=grid_id)
        
        if not grid_data:
            status_display.value = "<b>Status:</b> <span style='color: red;'>⚠️ Failed to load grid</span>"
            return
        
        # Extract data from dictionary
        signal_arrays = grid_data.get('signal_arrays', {})
        metadata = grid_data.get('metadata', {})
        grid_name = grid_data.get('grid_name', 'Unknown')
        
        # Store loaded data
        loaded_grid_data = {
            'grid_data': grid_data,
            'metadata': metadata,
            'signal_arrays': signal_arrays
        }
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Loaded grid: {grid_name} ({len(signal_arrays)} signal(s))</span>"
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()

# Function to update UI based on data source mode
def update_data_source_mode(change):
    """Show/hide MongoDB widgets based on data source mode."""
    if change['new'] == 'mongodb':
        model_dropdown.layout.display = 'flex'
        grid_type_filter.layout.display = 'flex'
        grid_dropdown.layout.display = 'flex'
        load_grid_button.layout.display = 'flex'
    else:
        model_dropdown.layout.display = 'none'
        grid_type_filter.layout.display = 'none'
        grid_dropdown.layout.display = 'none'
        load_grid_button.layout.display = 'none'

# Connect events
data_source_mode.observe(update_data_source_mode, names='value')
update_data_source_mode({'new': data_source_mode.value})
model_dropdown.observe(update_grid_dropdown, names='value')
grid_type_filter.observe(update_grid_dropdown, names='value')
load_grid_button.on_click(load_grid_from_mongodb)

left_panel = VBox([
    problem_section,
    sobol_section,
    morris_section,
    local_section,
    doe_section,
    uq_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Indices', 'indices'), ('Rankings', 'rankings'), ('Comparison', 'comparison'), ('Uncertainty', 'uncertainty')],
    value='indices',
    description='View:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Sensitivity Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results
# ============================================

# Sensitivity Indices
indices_label = widgets.HTML("<b>Sensitivity Indices:</b>")
indices_display = widgets.HTML("No analysis performed yet")
indices_section = VBox([
    indices_label,
    indices_display
], layout=Layout(padding='5px'))

# Parameter Rankings
rankings_label = widgets.HTML("<b>Parameter Rankings:</b>")
rankings_display = widgets.HTML("No rankings available")
rankings_section = VBox([
    rankings_label,
    rankings_display
], layout=Layout(padding='5px'))

# Method Performance
performance_label = widgets.HTML("<b>Method Performance:</b>")
performance_display = widgets.HTML("No performance data")
performance_section = VBox([
    performance_label,
    performance_display
], layout=Layout(padding='5px'))

# Uncertainty Results
uncertainty_label = widgets.HTML("<b>Uncertainty Results:</b>")
uncertainty_display = widgets.HTML("No uncertainty results")
uncertainty_section = VBox([
    uncertainty_label,
    uncertainty_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_indices_button = Button(description='Export Indices', button_style='', layout=Layout(width='150px'))
export_rankings_button = Button(description='Export Rankings', button_style='', layout=Layout(width='150px'))
export_report_button = Button(description='Export Report', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_indices_button,
    export_rankings_button,
    export_report_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    indices_section,
    rankings_section,
    performance_section,
    uncertainty_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to analyze")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
info_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    progress_bar,
    info_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Analysis Functions
# ============================================

def execute_analysis(button):
    """Execute sensitivity analysis based on current settings."""
    global sensitivity_results, loaded_grid_data, signal_arrays
    
    status_display.value = "<b>Status:</b> Analyzing sensitivity..."
    progress_bar.value = 0
    info_display.value = ""
    
    try:
        method = sensitivity_method.value
        progress_bar.value = 20
        
        # Check data source and prepare data
        if data_source_mode.value == 'mongodb':
            if not loaded_grid_data or not signal_arrays:
                status_display.value = "<b>Status:</b> <span style='color: red;'>⚠️ Please load a grid from MongoDB first</span>"
                return
            
            # Extract metadata for process parameters (if available)
            metadata = loaded_grid_data.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # For sensitivity analysis, we can use:
            # - Process parameters from metadata as inputs
            # - Quality scores or signal values as outputs
            # This is a simplified version - in practice, you'd need multiple builds with different parameters
            
            # Use demo data for now if real parameter data isn't available
            # In a real scenario, you'd collect data from multiple builds
            if not config_meta.get('process_parameters'):
                status_display.value = "<b>Status:</b> <span style='color: orange;'>⚠️ Process parameters not found. Using sample data for sensitivity analysis.</span>"
                # Continue with demo data
            else:
                # Extract real process parameters
                process_params = config_meta.get('process_parameters', {})
                # Use these for sensitivity analysis
                pass
        
        progress_bar.value = 30
        
        # Perform analysis based on method
        if method == 'sobol':
            results = perform_sobol_analysis()
        elif method == 'morris':
            results = perform_morris_analysis()
        elif method == 'local':
            results = perform_local_analysis()
        elif method == 'doe':
            results = perform_doe_analysis()
        elif method == 'uq':
            results = perform_uq_analysis()
        else:
            results = {}
        
        sensitivity_results = results
        progress_bar.value = 80
        
        # Update displays
        update_results_display()
        update_visualization()
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Analysis completed</span>"
        output_type = output_model_selector.value if data_source_mode.value == 'mongodb' else 'quality'
        info_display.value = f"<p>Method: <b>{method}</b> | Output: <b>{output_type}</b></p>"
        
    except Exception as e:
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during analysis</span>"
        progress_bar.value = 0

def perform_sobol_analysis():
    """Perform Sobol sensitivity analysis (simplified demo)."""
    np.random.seed(42)
    
    # Generate samples
    n_samples = sobol_sample_size.value
    n_vars = len(variable_names)
    
    # Generate random samples
    samples = {}
    for var_name in variable_names:
        bounds = variable_bounds[var_name]
        samples[var_name] = np.random.uniform(bounds[0], bounds[1], n_samples)
    
    # Evaluate model
    outputs = []
    for i in range(n_samples):
        params = {var: samples[var][i] for var in variable_names}
        outputs.append(demo_model_function(params))
    outputs = np.array(outputs)
    
    # Simplified Sobol indices calculation
    # In real implementation, would use Saltelli sampling
    s1_indices = {}
    st_indices = {}
    
    for var_name in variable_names:
        # Simplified: correlation-based approximation
        var_data = samples[var_name]
        corr = np.corrcoef(var_data, outputs)[0, 1]
        s1_indices[var_name] = abs(corr) ** 2
        st_indices[var_name] = abs(corr) ** 2 + 0.1  # Add interaction term
    
    return {
        'method': 'Sobol',
        's1_indices': s1_indices,
        'st_indices': st_indices,
        's2_indices': {} if not sobol_second_order.value else {'laser_power-scan_speed': 0.05},
        'variables': variable_names,
        'sample_size': n_samples
    }

def perform_morris_analysis():
    """Perform Morris sensitivity analysis (simplified demo)."""
    np.random.seed(42)
    
    n_trajectories = morris_sample_size.value // (len(variable_names) + 1)
    levels = morris_levels.value
    
    # Simplified Morris metrics
    mu_star = {}
    sigma = {}
    
    for var_name in variable_names:
        # Simplified calculation
        mu_star[var_name] = np.random.uniform(0.1, 0.8)
        sigma[var_name] = np.random.uniform(0.05, 0.3)
    
    return {
        'method': 'Morris',
        'mu_star': mu_star,
        'sigma': sigma,
        'variables': variable_names,
        'n_trajectories': n_trajectories
    }

def perform_local_analysis():
    """Perform local sensitivity analysis."""
    # Baseline parameters
    baseline = {
        'laser_power': 225.0,
        'scan_speed': 1.2,
        'layer_thickness': 0.05
    }
    
    baseline_output = demo_model_function(baseline)
    perturbation = local_perturbation.value
    
    derivatives = {}
    for var_name in variable_names:
        params_perturbed = baseline.copy()
        bounds = variable_bounds[var_name]
        param_range = bounds[1] - bounds[0]
        params_perturbed[var_name] += perturbation * param_range
        
        perturbed_output = demo_model_function(params_perturbed)
        derivative = (perturbed_output - baseline_output) / (perturbation * param_range)
        derivatives[var_name] = derivative
    
    return {
        'method': 'Local',
        'derivatives': derivatives,
        'baseline': baseline,
        'baseline_output': baseline_output
    }

def perform_doe_analysis():
    """Perform Design of Experiments analysis."""
    np.random.seed(42)
    
    n_samples = doe_sample_size.value
    
    # Generate samples based on design type
    samples = {}
    for var_name in variable_names:
        bounds = variable_bounds[var_name]
        if doe_design_type.value == 'lhs':
            # Latin Hypercube Sampling
            samples[var_name] = np.linspace(bounds[0], bounds[1], n_samples)
            np.random.shuffle(samples[var_name])
        else:
            samples[var_name] = np.random.uniform(bounds[0], bounds[1], n_samples)
    
    # Evaluate model
    outputs = []
    for i in range(n_samples):
        params = {var: samples[var][i] for var in variable_names}
        outputs.append(demo_model_function(params))
    
    return {
        'method': 'DoE',
        'design_type': doe_design_type.value,
        'samples': samples,
        'outputs': outputs,
        'n_samples': n_samples
    }

def perform_uq_analysis():
    """Perform uncertainty quantification analysis."""
    np.random.seed(42)
    
    n_samples = uq_sample_size.value
    
    # Generate samples with uncertainty
    samples = {}
    for var_name in variable_names:
        bounds = variable_bounds[var_name]
        if uq_distribution.value == 'normal':
            mean = (bounds[0] + bounds[1]) / 2
            std = (bounds[1] - bounds[0]) / 6
            samples[var_name] = np.random.normal(mean, std, n_samples)
            samples[var_name] = np.clip(samples[var_name], bounds[0], bounds[1])
        else:
            samples[var_name] = np.random.uniform(bounds[0], bounds[1], n_samples)
    
    # Evaluate model
    outputs = []
    for i in range(n_samples):
        params = {var: samples[var][i] for var in variable_names}
        outputs.append(demo_model_function(params))
    outputs = np.array(outputs)
    
    return {
        'method': 'UQ',
        'uq_method': uq_method.value,
        'output_mean': np.mean(outputs),
        'output_std': np.std(outputs),
        'output_ci': (np.percentile(outputs, 2.5), np.percentile(outputs, 97.5)),
        'outputs': outputs
    }

def update_results_display():
    """Update results displays."""
    global sensitivity_results
    
    if not sensitivity_results:
        return
    
    method = sensitivity_method.value
    
    # Sensitivity indices
    if method == 'sobol' and 's1_indices' in sensitivity_results:
        indices_html = "<table border='1' style='border-collapse: collapse; width: 100%;'><tr><th>Variable</th><th>S1</th><th>ST</th></tr>"
        for var in variable_names:
            s1 = sensitivity_results['s1_indices'].get(var, 0)
            st = sensitivity_results['st_indices'].get(var, 0)
            indices_html += f"<tr><td>{var}</td><td>{s1:.4f}</td><td>{st:.4f}</td></tr>"
        indices_html += "</table>"
        indices_display.value = indices_html
    
    # Parameter rankings
    if method == 'sobol' and 's1_indices' in sensitivity_results:
        rankings = sorted(sensitivity_results['s1_indices'].items(), key=lambda x: x[1], reverse=True)
        rankings_html = "<ol>"
        for rank, (var, value) in enumerate(rankings, 1):
            rankings_html += f"<li><b>{var}:</b> {value:.4f}</li>"
        rankings_html += "</ol>"
        rankings_display.value = rankings_html
    elif method == 'morris' and 'mu_star' in sensitivity_results:
        rankings = sorted(sensitivity_results['mu_star'].items(), key=lambda x: x[1], reverse=True)
        rankings_html = "<ol>"
        for rank, (var, value) in enumerate(rankings, 1):
            rankings_html += f"<li><b>{var}:</b> μ*={value:.4f}</li>"
        rankings_html += "</ol>"
        rankings_display.value = rankings_html
    
    # Method performance
    if 'sample_size' in sensitivity_results:
        performance_html = f"<p><b>Sample Size:</b> {sensitivity_results['sample_size']:,}</p>"
        performance_html += f"<p><b>Computation Time:</b> ~{sensitivity_results['sample_size'] * 0.001:.2f} s</p>"
        performance_display.value = performance_html
    
    # Uncertainty results
    if method == 'uq' and 'output_mean' in sensitivity_results:
        uq_html = f"<p><b>Mean:</b> {sensitivity_results['output_mean']:.4f}</p>"
        uq_html += f"<p><b>Std:</b> {sensitivity_results['output_std']:.4f}</p>"
        ci = sensitivity_results['output_ci']
        uq_html += f"<p><b>95% CI:</b> [{ci[0]:.4f}, {ci[1]:.4f}]</p>"
        uncertainty_display.value = uq_html

def update_visualization():
    """Update visualization display."""
    global sensitivity_results
    
    with viz_output:
        clear_output(wait=True)
        
        if not sensitivity_results:
            display(HTML("<p>Execute analysis to see visualization</p>"))
            return
        
        method = sensitivity_method.value
        mode = viz_mode.value
        
        if mode == 'indices':
            if method == 'sobol' and 's1_indices' in sensitivity_results:
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                # S1 indices bar chart
                vars_list = list(sensitivity_results['s1_indices'].keys())
                s1_values = [sensitivity_results['s1_indices'][v] for v in vars_list]
                st_values = [sensitivity_results['st_indices'][v] for v in vars_list]
                
                x = np.arange(len(vars_list))
                width = 0.35
                axes[0].bar(x - width/2, s1_values, width, label='S1 (First Order)', alpha=0.8)
                axes[0].bar(x + width/2, st_values, width, label='ST (Total Order)', alpha=0.8)
                axes[0].set_xlabel('Variables')
                axes[0].set_ylabel('Sensitivity Index')
                axes[0].set_title('Sobol Sensitivity Indices')
                axes[0].set_xticks(x)
                axes[0].set_xticklabels(vars_list, rotation=45)
                axes[0].legend()
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Pie chart of contributions
                axes[1].pie(s1_values, labels=vars_list, autopct='%1.1f%%', startangle=90)
                axes[1].set_title('Contribution to Variance')
                
                plt.tight_layout()
                plt.show()
            
            elif method == 'morris' and 'mu_star' in sensitivity_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                
                vars_list = list(sensitivity_results['mu_star'].keys())
                mu_star_values = [sensitivity_results['mu_star'][v] for v in vars_list]
                sigma_values = [sensitivity_results['sigma'][v] for v in vars_list]
                
                ax.scatter(mu_star_values, sigma_values, s=100, alpha=0.6)
                for i, var in enumerate(vars_list):
                    ax.annotate(var, (mu_star_values[i], sigma_values[i]))
                ax.set_xlabel('μ* (Mean Absolute Effect)')
                ax.set_ylabel('σ (Standard Deviation)')
                ax.set_title('Morris Screening Results')
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            
            elif method == 'local' and 'sensitivities' in sensitivity_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                
                vars_list = list(sensitivity_results['sensitivities'].keys())
                sens_values = [sensitivity_results['sensitivities'][v] for v in vars_list]
                
                colors = ['green' if v > 0 else 'red' for v in sens_values]
                ax.bar(vars_list, sens_values, color=colors, alpha=0.7)
                ax.set_xlabel('Variables')
                ax.set_ylabel('Sensitivity (Derivative)')
                ax.set_title('Local Sensitivity Indices')
                ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
                ax.set_xticklabels(vars_list, rotation=45)
                ax.grid(True, alpha=0.3, axis='y')
                plt.tight_layout()
                plt.show()
            
            elif method == 'doe' and 'outputs' in sensitivity_results:
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                samples = sensitivity_results.get('samples', {})
                outputs = sensitivity_results['outputs']
                
                # Output distribution
                axes[0].hist(outputs, bins=30, alpha=0.7, edgecolor='black')
                axes[0].axvline(np.mean(outputs), color='r', linestyle='--', label=f'Mean: {np.mean(outputs):.3f}')
                axes[0].set_xlabel('Output Value')
                axes[0].set_ylabel('Frequency')
                axes[0].set_title('Output Distribution (DoE)')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Parameter vs Output scatter (first parameter)
                if samples and len(outputs) > 0:
                    first_var = list(samples.keys())[0] if samples else None
                    if first_var:
                        axes[1].scatter(samples[first_var], outputs, alpha=0.5)
                        axes[1].set_xlabel(f'{first_var}')
                        axes[1].set_ylabel('Output')
                        axes[1].set_title(f'Output vs {first_var}')
                        axes[1].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
            
            else:
                display(HTML(f"<p>Indices visualization not available for {method} method</p>"))
        
        elif mode == 'rankings':
            if method == 'sobol' and 's1_indices' in sensitivity_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                
                rankings = sorted(sensitivity_results['s1_indices'].items(), key=lambda x: x[1], reverse=True)
                vars_list = [r[0] for r in rankings]
                values = [r[1] for r in rankings]
                
                ax.barh(vars_list, values, color='steelblue')
                ax.set_xlabel('Sensitivity Index')
                ax.set_title('Parameter Rankings (Sobol S1)')
                ax.grid(True, alpha=0.3, axis='x')
                plt.tight_layout()
                plt.show()
            
            elif method == 'morris' and 'mu_star' in sensitivity_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                
                rankings = sorted(sensitivity_results['mu_star'].items(), key=lambda x: x[1], reverse=True)
                vars_list = [r[0] for r in rankings]
                values = [r[1] for r in rankings]
                
                ax.barh(vars_list, values, color='coral')
                ax.set_xlabel('μ* (Mean Absolute Effect)')
                ax.set_title('Parameter Rankings (Morris μ*)')
                ax.grid(True, alpha=0.3, axis='x')
                plt.tight_layout()
                plt.show()
            
            elif method == 'local' and 'sensitivities' in sensitivity_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                
                rankings = sorted(sensitivity_results['sensitivities'].items(), key=lambda x: abs(x[1]), reverse=True)
                vars_list = [r[0] for r in rankings]
                values = [r[1] for r in rankings]
                
                colors = ['green' if v > 0 else 'red' for v in values]
                ax.barh(vars_list, values, color=colors, alpha=0.7)
                ax.set_xlabel('Sensitivity (Derivative)')
                ax.set_title('Parameter Rankings (Local Sensitivity)')
                ax.axvline(0, color='black', linestyle='-', linewidth=0.5)
                ax.grid(True, alpha=0.3, axis='x')
                plt.tight_layout()
                plt.show()
            
            elif method == 'doe' and 'outputs' in sensitivity_results:
                # For DoE, rank by output variance contribution
                fig, ax = plt.subplots(figsize=(10, 6))
                
                # Calculate variance contribution for each parameter
                samples = sensitivity_results.get('samples', {})
                outputs = sensitivity_results['outputs']
                
                if samples and len(outputs) > 0:
                    variances = {}
                    for var_name in variable_names:
                        if var_name in samples:
                            var_values = samples[var_name]
                            # Simple correlation-based ranking
                            if len(var_values) == len(outputs):
                                corr = np.abs(np.corrcoef(var_values, outputs)[0, 1])
                                variances[var_name] = corr if not np.isnan(corr) else 0
                    
                    if variances:
                        rankings = sorted(variances.items(), key=lambda x: x[1], reverse=True)
                        vars_list = [r[0] for r in rankings]
                        values = [r[1] for r in rankings]
                        
                        ax.barh(vars_list, values, color='purple', alpha=0.7)
                        ax.set_xlabel('Correlation with Output')
                        ax.set_title('Parameter Rankings (DoE)')
                        ax.grid(True, alpha=0.3, axis='x')
                        plt.tight_layout()
                        plt.show()
            
            else:
                display(HTML(f"<p>Rankings visualization not available for {method} method</p>"))
        
        elif mode == 'comparison':
            # Compare multiple methods if results are available
            fig, ax = plt.subplots(figsize=(12, 6))
            
            # Collect rankings from different methods
            all_rankings = {}
            
            if 's1_indices' in sensitivity_results:
                sobol_rankings = sorted(sensitivity_results['s1_indices'].items(), key=lambda x: x[1], reverse=True)
                for rank, (var, val) in enumerate(sobol_rankings, 1):
                    if var not in all_rankings:
                        all_rankings[var] = {}
                    all_rankings[var]['Sobol'] = rank
            
            if 'mu_star' in sensitivity_results:
                morris_rankings = sorted(sensitivity_results['mu_star'].items(), key=lambda x: x[1], reverse=True)
                for rank, (var, val) in enumerate(morris_rankings, 1):
                    if var not in all_rankings:
                        all_rankings[var] = {}
                    all_rankings[var]['Morris'] = rank
            
            if all_rankings:
                vars_list = list(all_rankings.keys())
                methods = list(set([m for v in all_rankings.values() for m in v.keys()]))
                
                x = np.arange(len(vars_list))
                width = 0.8 / len(methods)
                
                for i, method in enumerate(methods):
                    ranks = [all_rankings[v].get(method, len(vars_list) + 1) for v in vars_list]
                    ax.bar(x + i * width, ranks, width, label=method, alpha=0.7)
                
                ax.set_xlabel('Variables')
                ax.set_ylabel('Rank')
                ax.set_title('Method Comparison (Lower Rank = Higher Sensitivity)')
                ax.set_xticks(x + width * (len(methods) - 1) / 2)
                ax.set_xticklabels(vars_list, rotation=45)
                ax.legend()
                ax.grid(True, alpha=0.3, axis='y')
                ax.invert_yaxis()  # Lower rank (1) at top
                plt.tight_layout()
                plt.show()
            else:
                display(HTML("<p>Execute multiple methods to compare results</p>"))
        
        elif mode == 'uncertainty':
            if method == 'uq' and 'outputs' in sensitivity_results:
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                outputs = sensitivity_results['outputs']
                
                # Distribution
                axes[0].hist(outputs, bins=50, alpha=0.7, edgecolor='black')
                axes[0].axvline(sensitivity_results['output_mean'], color='r', linestyle='--', label='Mean')
                axes[0].axvline(sensitivity_results['output_ci'][0], color='orange', linestyle='--', label='95% CI')
                axes[0].axvline(sensitivity_results['output_ci'][1], color='orange', linestyle='--')
                axes[0].set_xlabel('Output Value')
                axes[0].set_ylabel('Frequency')
                axes[0].set_title('Output Distribution')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Box plot
                axes[1].boxplot([outputs], vert=True)
                axes[1].set_ylabel('Output Value')
                axes[1].set_title('Output Statistics')
                axes[1].grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            elif method == 'doe' and 'outputs' in sensitivity_results:
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                outputs = sensitivity_results['outputs']
                
                # Distribution
                axes[0].hist(outputs, bins=30, alpha=0.7, edgecolor='black')
                axes[0].axvline(np.mean(outputs), color='r', linestyle='--', label=f'Mean: {np.mean(outputs):.3f}')
                axes[0].axvline(np.percentile(outputs, 2.5), color='orange', linestyle='--', label='95% CI')
                axes[0].axvline(np.percentile(outputs, 97.5), color='orange', linestyle='--')
                axes[0].set_xlabel('Output Value')
                axes[0].set_ylabel('Frequency')
                axes[0].set_title('Output Distribution (DoE)')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Box plot
                axes[1].boxplot([outputs], vert=True)
                axes[1].set_ylabel('Output Value')
                axes[1].set_title('Output Statistics (DoE)')
                axes[1].grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            elif method in ['sobol', 'morris', 'local']:
                display(HTML(f"<p>Uncertainty visualization available for UQ and DoE methods. Current method: {method}</p>"))
            
            else:
                display(HTML(f"<p>Uncertainty visualization not available for {method} method</p>"))

# Connect events
execute_button.on_click(execute_analysis)
viz_mode.observe(lambda x: update_visualization(), names='value')
sensitivity_method.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to perform sensitivity analysis for process optimization.

### Key Takeaways

1. **Global Methods**: Sobol indices (S1, ST, S2), Morris screening (μ*, σ), FAST
2. **Local Methods**: Derivatives, gradients, perturbation analysis
3. **Design of Experiments**: Factorial, LHS, Random, Grid designs
4. **Uncertainty Quantification**: Monte Carlo, Bayesian, Taylor expansion
5. **Parameter Ranking**: Identify most influential parameters
6. **Visualization**: Sensitivity indices, rankings, uncertainty distributions

### Next Steps

Proceed to:
- **11_Process_Analysis_and_Optimization.ipynb** - Learn process analysis and optimization
- **12_Anomaly_Detection.ipynb** - Learn anomaly detection techniques

### Related Resources

- Sensitivity Analysis Documentation: `../docs/AM_QADF/05-modules/analytics.md`
- API Reference: `../docs/AM_QADF/06-api-reference/analytics-api.md`
- Examples: `../examples/`
